# PS.3 (triangular solver)

$Ly=b$ with $L$ lower triangular with $1$ in the diagonal.

\begin{align*}
y_1 &= b_1,\\
y_2 &= b_2-\ell_{2,1}y_1,\\
&\vdots \\
y_k & = b_k - \sum_{j=1}^{k-1}\ell_{k,j}y_j,  \\
&\vdots \\
y_n&=b_n-\sum_{j=1}^{n-1}\ell_{n,j}y_j.
\end{align*}

a) Write a solver for $Ly=b$.

In [44]:
function myL(L,b)
    n = length(b)
    y = copy(b) 
    for k = 2:n
        y[k] = b[k] - sum(L[k, j] * y[j] for j in 1:(k-1))
    end
    return y
end

myL (generic function with 1 method)

$Ux=y$ with $U$ upper triangular and nonzero diagonal.

\begin{align*}
x_n &= \frac{y_n}{u_{n,n}}\\
x_{n-1} &= \frac{1}{u_{n-1,n-1}} \left(y_{n-1}-u_{n-1,n}x_{n}\right)\\
&\vdots\\
 x_{k}&=\frac{1}{u_{k,k}}\left(y_{k}-\sum_{j=k+1}^{n}u_{k,j}x_j\right) \\
 &\vdots\\
x_1&=\frac{1}{u_{1,1}}\left(y_1-\sum_{j=2}^{n}u_{1,j}x_j\right).
\end{align*}

b) Write a solver for $Ux=y$.

In [68]:
function myU(U,y)
    n = length(y)
    x = copy(y)
    for k = 0:(n-1)
        x[n-k] = (y[n-k] - sum(U[n-k, j] for j in (n-k+1):n; init = 0)) / U[n-k, n-k]
    end
    return x
end    

myU (generic function with 1 method)

# PS.3 (lu-decomposition without pivoting)

For $A\in \mathbb R^{n\times n}$, set $A^{(1)}:=B^{(1)}:=A$ and
\begin{align}
\ell^{(k)} = \frac{1}{a^{(k)}_{k,k}}\begin{pmatrix}
a^{(k)}_{k+1,k}\\
\vdots\\
a^{(k)}_{n,k}
\end{pmatrix}\in \mathbb R^{n-k}, \qquad\qquad
B^{(k+1)} = B^{(k)} - \ell^{(k)} \cdot \left(a^{(k)}_{k,k+1} , \cdots , a^{(k)}_{k,n}\right)\in \mathbb{R}^{(n-k)\times (n-k)}\\
A^{(k+1)}= L_k A^{(k)} = \begin{pmatrix}
a^{(1)}_{1,1} & a^{1)}_{1,2} &\cdots &\cdots& a^{(1)}_{1,n}\\
0 & \ddots & \\
\vdots&\ddots & a^{(k)}_{k,k} & \cdots & a^{(k)}_{k,n}\\
0 &&0 \\
\vdots &&\vdots&B^{(k+1)}\\
0 &\cdots &0
\end{pmatrix},\qquad
L = 
\begin{pmatrix}
1& 0 &\cdots &\cdots&\cdots&0\\
|& \ddots&\ddots&&&\vdots \\
| & &\ddots&\ddots&&\vdots\\
|&&|&\ddots&0&\vdots\\
|&&|&&1&0\\
\ell^{(1)} & \cdots&\ell^{(k)}& \cdots & \ell^{(n-1)}&1
\end{pmatrix},\quad U = A^{(n)}
\end{align}

We may store $\ell^{(1)},\ldots,\ell^{(k)}$ in $A^{(k+1)}$ to get rid of $L$, i.e., 
\begin{equation}
\tilde{A}^{(k+1)}=  \begin{pmatrix}
a^{(1)}_{1,1} & a^{1)}_{1,2} &\cdots &\cdots& a^{(1)}_{1,n}\\
| & \ddots & \\
|&\ddots & a^{(k)}_{k,k} & \cdots & a^{(k)}_{k,n}\\
| &&| \\
| &&|&B^{(k+1)}\\
\ell^{(1)} &\cdots &\ell^{(k)}
\end{pmatrix}
\end{equation}

a) Write a function with input $A$ that returns L and U of the lu-decomposition.

In [108]:
using LinearAlgebra

function myLU(A0)
    n = size(A0,1)
    A = copy(A0)
    for k in 1:n-1
        l = A[k+1:n,k]/A[k,k]
        A[k+1:n,k] = l
        # tick makes Julia aware that this is l * A[k, k+1:n].T
        A[k+1:n,k+1:n] = A[k+1:n,k+1:n] - l * A[k, k+1:n]' 
    end
    return tril(A,-1)+I, triu(A)
end

myLU (generic function with 1 method)

b) Solve the system of linear equations $Ax=b$ for 

\begin{equation}
A = \begin{pmatrix} \varepsilon & 1\\
1 & 1
\end{pmatrix},\qquad b =\begin{pmatrix} 1 \\ 2\end{pmatrix}
\end{equation}

by using myLU and myU, myL for $\varepsilon=2$ and for $\varepsilon = 2^{-55}$. Are the results correct?

In [105]:
A = [2. 1.; 1. 1.]
b = [1., 2.]
L, U = myLU(A)
y = myL(L, b)
x = myU(U, y)

2-element Vector{Float64}:
 0.0
 3.0

In [106]:
A = [2^-55 1.; 1. 1.]
L, U = myLU(A)
y = myL(L, b)
x = myU(U, y)

2-element Vector{Float64}:
 0.0
 1.0

# PS.3 (lu-decomposition with partial pivoting)

\begin{equation}
PA = LU,\qquad \qquad\qquad Ax=b \quad \Leftrightarrow \quad LUx=Pb
\end{equation}

a) Solve the system of linear equations $Ax=b$ for 

\begin{equation}
A = \begin{pmatrix} -2&2& -1\\
6 & -6 & 7\\
3&-8&4
\end{pmatrix},\qquad b =\begin{pmatrix} -1 \\ 7\\ -1\end{pmatrix}
\end{equation}

by using the lu-decomposition with partial pivoting by hand. 

b) The function myLUpivot is provided below. Write a function myLinearSolver that solves $Ax=b$ based on myLUpivot. Solve part a) with your implementation. Run testset and, if necessary, adjust the error bounds. 

In [109]:
function myLUpivot(A0)
    n = size(A0,1)
    p = [1:n;]
    A = copy(A0)
    for k in 1:n-1
        mptr = argmax(abs.(A[p[k:n],k]))
        p[k],p[k-1+mptr] = p[k-1+mptr],p[k]
        l = A[p[k+1:n],k]/A[p[k],k]
        A[p[k+1:n],k] .= l
        A[p[k+1:n],k+1:n] .= A[p[k+1:n],k+1:n]-l*transpose(A[p[k],k+1:n])
    end
    return tril(A[p,:],-1)+I, triu(A[p,:]), p
end

myLUpivot (generic function with 1 method)

In [110]:
function myLinearSolver(A,b)
    L, U = myLUpivot(A)
    y = myL(L, b)
    return myU(U, y)
end

myLinearSolver (generic function with 1 method)

In [ ]:
using Test
@testset "myLU" begin     
    for n in 1:100
        A = randn(n,n)
        b = randn(n)
        L,U = myLU(A)
        y = myL(L,b) 
        x = myU(U,y)
        @test norm(L*U-A) < cond(A) *1e-10 
        @test norm(A*x-b)/norm(A) < cond(A) *1e-10 
        
        L,U,p = myLUpivot(A)
        x = myLinearSolver(A,b)
        @test norm(L*U-A[p,:]) < cond(A) *1e-10 
        @test norm(A*x-b)/norm(A) < cond(A) *1e-10 
    end
end

# PS.3 (Cholesky factorization)

For symmetric, positive definite $A=LL^\top$, the matrix $L=(l_{i,j})$ is given by 

\begin{align}
l_{j,j} & = \sqrt{a_{j,j}-\sum_{k=1}^{j-1} l_{j,k}^2},& j&=1,\ldots,n,  \\
l_{i,j} & = \frac{1}{l_{j,j}} \left(a_{i,j}-\sum_{k=1}^{j-1} l_{i,k}l_{j,k} \right),& i&=j+1,\ldots,n.
\end{align}

a) Write a function with input $A$ that returns $L$.

In [128]:
function myCholesky(A)
    n = size(A,1)
    L = zeros(n,n)
    for j in 1:n
        L[j,j] = sqrt(A[j,j] - sum(L[j,k]^2 for k in 1:(j-1); init = 0))
        for i in (j+1):n
            L[i,j] = (A[i,j] - sum(L[i,k] * L[j,k] for k in 1:(j-1); init = 0)) / L[j,j]
        end
    end
    return L
end

myCholesky (generic function with 1 method)

b) Complete the testset "myCholesky" and run the tests successfully.

In [141]:
using Test
@testset "myCholesky" begin     
    for n in 1:20
        ℒ = randn(n,n)
        A = ℒ*ℒ'
        L = myCholesky(A)
        b = rand(n)
        y = L\b
        x = L'\y
        # somehow do this with the eps function
        @test norm(L*L'-A)/norm(A) <= 1^-15
        @test norm(A*x-b) <= 1^-15
    end
end

Test Summary: | Pass  Total  Time
myCholesky    |   40     40  0.0s


Test.DefaultTestSet("myCholesky", Any[], 40, false, false, true, 1.667744030755477e9, 1.667744030755651e9)

# PS.3 (lls via normal equations)

Given data $\alpha,b\in\mathbb{R}^m$, consider 

\begin{equation*}
\arg\min_{x\in\mathbb{R}^2}\sum_{k=1}^m \left|b_k-(x_1+x_2\alpha_k) \right|^2
\end{equation*}

The minimizer $(x_1,x_2)$ defines an affine linear function $f(a)=x_1+x_2a$.

a) Write a julia function that takes input $\alpha, b$ and returns $f$. 

In [143]:
using LinearAlgebra
function myLLS(α,b)
    A = zeros(length(α), 2)
    A[:, 2] = α
    L = myCholesky(A' * A)
    y = L\(A' * b)
    x = L'\y
    return a->x[1]+x[2]*a 
end

myLLS (generic function with 1 method)

b) Generate several data sets with illustrative plots.

In [ ]:
# α = ...
# b = ...
# ...

# using Plots
# scatter(α,b,xlabel="α",ylabel="b",label="data")
# scatter!(α,f.(α),label="lls")
# plot!(f,label="lls line")

# PS.3 (svd)

For $m\geq n$, let $A\in\mathbb{R}^{m\times n}$ with $rank(A)=p$. 

- The matrix $A^\top A\in\mathbb{R}^{n\times n}$ has eigenvalues $\lambda_1\geq \ldots\geq \lambda_p>0$ with pairwise orthonormal eigenvectors $v_1,\ldots,v_p\in\mathbb{R}^n$ that are collected into

\begin{equation*}
V:=(v_1,\ldots,v_p)\in\mathbb{R}^{n\times p}.
\end{equation*}


- The singular values $\sigma_k:=\sqrt{\lambda_k}$, for $k=1,\ldots,p$, are collected into

\begin{equation*}
 \Sigma:=diag(\sigma_1,\ldots,\sigma_p)\in\mathbb{R}^{p\times p}.
\end{equation*}

- Define $u_k : = \frac{1}{\sigma_k} A v_k$, for $k=1,\ldots,p$, and build

\begin{equation*}
U:=(u_1,\ldots,u_p)\in\mathbb{R}^{m\times p}.
\end{equation*}


Then we obtain the decomposition

\begin{equation*}
A = U\Sigma V^\top.
\end{equation*}

The pseudo inverse of $A$ is $A^\# = V\Sigma^{-1}U^\top$. 

a) Write a julia function that takes input $A$ and returns $U$, $\sigma_1,\ldots,\sigma_p$, and $V$. 

In [190]:
function mySVD(A)
    p = rank(A)
    # eigenvalues sorted in ascending order
    F = eigen(A'*A)
    Fn = size(F.vectors, 2)
    V = zeros(size(A, 2), p)
    sigma = zeros(p, p)
    U = zeros(size(A, 1), p)
    for k in 1:p
        V[:,k] = F.vectors[:,Fn - k + 1]
        sigma[k,k] = sqrt(F.values[Fn - k + 1])
        U[:,k] = (A * V[:,k]) / sigma[k,k]
    end
    return U, sigma, V
end

mySVD (generic function with 1 method)

b) Write a julia function that takes input $A$ and returns $A^\#$. Eventually use $A^\#$ to compute $f$ in part a) of PS.3 (lls via normal equations).

In [192]:
function myPinv(A)
    U, sigma, V = mySVD(A)
    return V * inv(sigma) * U'
end

myPinv (generic function with 1 method)